# Functions

**Generate the B graph**

In [1]:
import numpy as np
from scipy.stats import multivariate_normal
from scipy.optimize import minimize
import random
from sklearn.metrics import f1_score
from time import time
import matplotlib.pyplot as plt
from numpy.ma.extras import average
from sklearn.metrics import confusion_matrix

In [2]:
SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
# plt.rcdefaults()

### Metrics

In [3]:
def kll(a, a_hat, sigma_x):
    ''' Computing the KL loss for the covariance matrices of Y'''
    p = len(a)
    B = B_mat_symmetric(a, p)
    B_hat = B_mat_symmetric(a_hat, p)
    sigma_y = cov_y(sigma_x, B)
    sigma_y_hat = cov_y(sigma_x, B_hat)
    theta_y_hat = np.linalg.solve(sigma_y_hat, np.eye(p))
    theta_y = np.linalg.solve(sigma_y, np.eye(p))
    return np.trace(sigma_y@theta_y_hat) - np.log(np.linalg.det(sigma_y@theta_y_hat)) - p

def rkll(a, a_hat, sigma_x):
    ''' Computing the reverse KL loss for the covariance matrices of Y'''
    return kll(a_hat, a, sigma_x)

def rte(a, a_hat, sigma_x):
    ''' Computing the relative trace error loss for the precision matrix'''
    p = len(a)
    B = B_mat_symmetric(a, p)
    B_hat = B_mat_symmetric(a_hat, p)
    sigma_y = cov_y(sigma_x, B)
    sigma_y_hat = cov_y(sigma_x, B_hat)
    theta_y_hat = np.linalg.solve(sigma_y_hat, np.eye(p))
    theta_y = np.linalg.solve(sigma_y, np.eye(p))
    return abs(1 - (np.trace(theta_y_hat)/np.trace(theta_y)))

def norm_loss(a, a_hat, sigma_x):
    ''' Computing the norm loss for the precision matrix'''
    p = len(a)
    B = B_mat_symmetric(a, p)
    B_hat = B_mat_symmetric(a_hat, p)
    sigma_y = cov_y(sigma_x, B)
    sigma_y_hat = cov_y(sigma_x, B_hat)
    theta_y_hat = np.linalg.solve(sigma_y_hat, np.eye(p))
    theta_y = np.linalg.solve(sigma_y, np.eye(p))
    fro = np.linalg.norm(theta_y-theta_y_hat,ord='fro')
    spe = np.linalg.norm(theta_y-theta_y_hat,ord=2)
    l1 = np.linalg.norm(theta_y-theta_y_hat,ord=1)
    return fro, spe, l1

def scores(a, a_hat, threshold_value):
    ''' Computing the support recovery based scores for the precision matrix'''
    a_thr = threshold(a_hat, threshold_value=threshold_value) # Apply the threshold
    a_binary = binarize_matrix(a_thr)  # Change the output to a binary matrix to check for support recovery
    true_a_binary = binarize_matrix(a)  # True Support Vector
    cm = confusion_matrix(true_a_binary, a_binary)
    tn,fp,fn,tp = cm[0,0], cm[0,1], cm[1,0], cm[1,1]
    accuracy = (tn + tp)/np.sum(cm)
    specificity = (tn)/(tn + fp) # TN/(TN+FP)
    sensitivity = (tp)/(tp + fn) # TP/(TP+FN)
    MCC = ((tp*tn)-(fp*fn))/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    return accuracy, specificity, sensitivity, MCC, cm

### Data Generation Functions

In [4]:
def B_mat_symmetric(a, p):
    '''To generate a symmetric toeplitz matrix with a given vector 'a' '''
    # Vector 'a' can only be of length p (odd length indexing from 0 to p-1 and center at 0)
    # Initialize a zeros matrix of size nxn
    result = np.zeros((p, p))

    # Fill in the main diagonal with center element of a
    np.fill_diagonal(result, a[0])

    # Fill in sub/super-diagonals (below the main diagonal)
    for i in range(1, p):
        np.fill_diagonal(result[i:], -1*a[i]) # Sub-diagonals
        np.fill_diagonal(result[:,i:], -1*a[i]) # Super-diagonals
    return result


def B_mat_lowertriangular(a, p):
    '''To generate a lower triangular toeplitz matrix with a given vector 'a' '''
    # Vector 'a' can only be of length p (odd length indexing from 0 to p-1 and center at 0)
    # Initialize a zeros matrix of size nxn
    result = np.zeros((p, p))

    # Fill in the main diagonal with center element of a
    np.fill_diagonal(result, a[0])

    # Fill in sub/diagonals (below the main diagonal)
    for i in range(1, p):
        np.fill_diagonal(result[i:], -1*a[i]) # Sub-diagonals
    return result


def cov_x(p):
    """ Generate the covariance matrix for x"""
    sigma_x = np.eye(p)
    return sigma_x


def cov_y(sigma_x, B):
    """ Generate covariance matrix for Y i.e. (B^(-1).X.B^(-T)) since Y=B^(-1).X"""
    B_inv = np.linalg.solve(B, np.eye(B.shape[0]))
    sigma_y = B_inv@sigma_x@B_inv.T
    return sigma_y


def generate_y(mean_y, sigma_y, n_samples):
    """ Generate samples of Y"""
    y_samples = multivariate_normal.rvs(mean=mean_y, cov=sigma_y, size=n_samples)
    return y_samples
    
    
def samp_cov(y_samples):
    """ Generating sample covariance matrix """
    n_samples = len(y_samples)
    samp_cov_y = np.sum(np.array([np.outer(y_samples[i],y_samples[i]) for i in range(n_samples)]), axis=0)/n_samples 
    return samp_cov_y


def soft(x, y):
    return np.sign(x) * np.maximum(np.abs(x) - y, 0)


def threshold(vec, threshold_value=0.002):
    vec1 = vec.copy()
    vec1[np.abs(vec) < threshold_value] = 0
    return vec1


def binarize_matrix(matrix):
    '''Converts non-zero elements to 1 and keeps zeros as zeros.'''
    binary_matrix = np.where(matrix != 0, 1, 0)
    return binary_matrix


def compute_recovery_rate_numpy(a_binary, true_a_binary):
    joint_ones = np.sum((a_binary == 1) & (true_a_binary == 1))
    ones_true_a = np.sum(true_a_binary == 1)
    if ones_true_a == 0:
        return 0
    return joint_ones / ones_true_a


def create_sparse_vec_pos_def_2(dim, nonzeros, diag=20):
    ''' Function to create sparse vector(p x 1) which can generate symmetric +ve definite toeplitz matrix'''
    vec = np.zeros((dim,))
    vec[0] = diag
    nonzeros -= 1
    # selection of indicies with the non-zero entry (random selection)
    non_zero_indices = random.sample(range(1,dim),nonzeros)
    for idx in non_zero_indices:
        # Randomly assign values to non-zero entries 
        vec[idx] = random.randint(1,5)
    return vec


### Different functions for ADMM ###

def f1(a, S, sigma_x, rho, a2, mu1, regularized = False):
    """ Function evaluation for Newton method with regularization term based on ADMM
        Tr[S.Theta_y] - logdet(Theta_y) + rho/2*||a1 - a2 + mu1||_1"""
    p = S.shape[0]
    a.reshape((-1,))
    B = B_mat_symmetric(a, p)
    theta_x = np.linalg.solve(sigma_x, np.eye(p))
    if regularized:
        # Added [1:] since we don't want to regularize a0
        reg_term = (rho/2)*np.linalg.norm(a[1:]-a2+mu1) 
    else:
        reg_term = 0
    return np.trace(S@B.T@theta_x@B)-np.log(np.linalg.det(B.T@theta_x@B))+reg_term 


def f2(a, lam):
    """ Function evaluation of regularization term """
    return lam*np.linalg.norm(a[1:],1)
    
    
def f(a, S, sigma_x):
    a.reshape((-1,))
    B = B_mat_symmetric(a, S.shape[0])
    theta_x = np.linalg.solve(sigma_x, np.eye(S.shape[0]))
    return np.trace(S@B.T@sigma_x_inv@B) - np.log(np.linalg.det(B.T@sigma_x_inv@B))

### Gradient Computation

In [5]:
def grad_f(x, S, sigma_x, rho, a2, mu1, regularized = False):
    ''' Computing gradient for f1(x) = Tr(S B(x) T B(x)) + log(det(B(x) T B(x)))
        both regularized and unregularized, where B is symmetric'''
    p = S.shape[0] # Dimensions
    B = B_mat_symmetric(x, p) # Generating B matrix with a given x
    B_inv = np.linalg.solve(B, np.eye(p))
    theta_x = np.linalg.solve(sigma_x, np.eye(p))
    
    if regularized:
        k = np.zeros_like(x)
        k[1:] = x[1:]-a2+mu1
    else:
        k = np.zeros_like(x)
    
    gradient = np.zeros((p,))
    for i in range(p):
        Ti = np.zeros((p,p))
        if i==0:
            np.fill_diagonal(Ti, 1)
            gradient[i] = 2*np.trace((S@B@theta_x@Ti) - (B_inv@Ti))
        else:
            np.fill_diagonal(Ti[i:], -1) # Sub-diagonals
            np.fill_diagonal(Ti[:,i:], -1) # Super-diagonals
            gradient[i] = 2*np.trace((S@B@theta_x@Ti) - (B_inv@Ti)) + (rho*k[i])
    return gradient


def hessian_f(x, S, sigma_x, rho, regularized = False):
    ''' Computing Hessian for f1(x) = Tr(S B(x) T B(x)) + log(det(B(x) T B(x)))
        both regularized and unregularized, where B is symmetric'''
    p = S.shape[0]
    B = B_mat_symmetric(x, p)
    B_inv = np.linalg.solve(B, np.eye(p))  
    theta_x = np.linalg.solve(sigma_x, np.eye(p))
    
    hessian = np.zeros((p,p))
    for i in range(p):
        Ti = np.zeros((p,p))
        if i==0:
            np.fill_diagonal(Ti, 1)
        else:
            np.fill_diagonal(Ti[i:], -1) # Sub-diagonals
            np.fill_diagonal(Ti[:,i:], -1) # Super-diagonals
        for j in range(p):
            Tj = np.zeros((p,p))
            if j==0:
                np.fill_diagonal(Tj, 1)
            else:
                np.fill_diagonal(Tj[j:], -1) # Sub-diagonals
                np.fill_diagonal(Tj[:,j:], -1) # Super-diagonals
            hessian[i,j] = 2*(np.trace(S@Ti@theta_x@Tj) + np.trace(B_inv@Ti@B_inv@Tj))
    if regularized:
        reg = rho*np.eye(p)
        reg[0,0] = 0
        hessian+=reg
    return hessian

###  Newtons Method

In [6]:
def compute_newton_step(gradient, hessian):
    """
    Computes the Newton step and decrement.

    Parameters:
    gradient (np.array): Gradient vector.
    hessian (np.array): Hessian matrix.

    Returns:
    delta_x_nt (np.array): Newton step.
    lambd (float): Newton decrement.
    """
    delta_x_nt = -np.linalg.solve(hessian, gradient)  # Newton step
    lambd_sqr = np.dot(gradient.T, hessian@gradient)  # Newton decrement
    return delta_x_nt, lambd_sqr

def backtracking_line_search(f, gradient, x, delta_x_nt, S, sigma_x, rho, a2, mu1,
                             regularized=False, alpha=0.01, beta=0.5):
    """
    Backtracking line search to ensure sufficient decrease.

    Parameters:
    f (function): Function to minimize.
    gradient (np.array): Gradient vector.
    x (np.array): Current point.
    delta_x_nt (np.array): Newton step.
    alpha (float): Alpha parameter for backtracking.
    beta (float): Beta parameter for backtracking.

    Returns:
    t (float): Step size.
    """
    t = 1  # Start with full step size
    p = S.shape[0]
    x_ = x + t * delta_x_nt
    e_val, e_vec = np.linalg.eig(B_mat_symmetric(x_, p))
    i = 0
    while not np.all(e_val>0): 
        t *= beta # Reduce step size so that x + del_x is PSD
        x_ = x + t * delta_x_nt
        e_val, e_vec = np.linalg.eig(B_mat_symmetric(x_, p))
        i+=1
        if i>20:
            t = 0
            print('Backtracking line search (I) went beyond 20 iterations')
            break
    x_ = x + t * delta_x_nt
    i = 0
    while (f(x_,S,sigma_x,rho,a2,mu1,regularized=regularized)\
           >f(x,S,sigma_x,rho,a2,mu1,regularized=regularized)\
           + alpha*t*np.dot(gradient.T,delta_x_nt)):
        t *= beta  # Reduce step size
        x_ = x + t * delta_x_nt
        if (i>20) or (t==0):
            t = 0
            print('Backtracking line search (II) went beyond 20 iterations')
            break
    return t


def projection_Rp(x):
    y = np.maximum(x, np.zeros_like(x))
    return y


def newton_method(f, x0, S, sigma_x, rho, a2, mu1, epsilon=1e-6, alpha=0.01, beta=0.5,
                  regularized=False, backtracking=False, projection=False):
    """
    Implements Newton's method for optimization.

    Parameters:
    f (function): Function to minimize.
    grad_f (function): Gradient function.
    hessian_f (function): Hessian function.
    x0 (np.array): Initial point.
    epsilon (float): Stopping criterion for Newton decrement.
    alpha (float): Alpha parameter for backtracking.
    beta (float): Beta parameter for backtracking.

    Returns:
    x (np.array): Optimized point.
    """
    x = x0
    i = 0
    while True:
        gradient = grad_f(x, S, sigma_x, rho, a2, mu1, regularized=regularized)
        hessian = hessian_f(x, S, sigma_x, rho, regularized=regularized)
        
        # Compute Newton step and decrement
        delta_x_nt, lambd_sqr = compute_newton_step(gradient, hessian)
        
        # Stopping criterion
        if lambd_sqr / 2 <= epsilon:
            print('Stopping Criteria Met')
            print(lambd_sqr, delta_x_nt)
            print(i)
            break

        # Backtracking line search
        if backtracking:
            print("line search")
            t = backtracking_line_search(f, gradient, x, delta_x_nt, S, sigma_x, rho, a2, mu1,
                                         regularized=regularized, alpha=alpha, beta=beta)
        else:
            t = 1
#         print(t)
            
        # Update x
        x_old = x.copy()
        x = x + t*delta_x_nt

        # Project x on R+
        if projection:
            x = projection_Rp(x)
        
        if i%10==0:
            print(lambd_sqr, delta_x_nt)
            print(i)
            
        if (i>=50):
            print('Newton algorithm finished 50 iterations')
            break
        
        if (i>1) and (np.linalg.norm(x_old-x)<=epsilon):
            print('not much change in x observed')
            print(i)
            break
        i+=1
    return x


### Pseudo Version of ADMM (Since f1 is not in closed form)

#### ADMM

In [7]:
def ADMM_newton(f, iterations, rho, lambda_param, S, sigma_x, a, epsilon=1e-6,
                alpha=0.5, beta=0.7, regularized=False, backtracking=False,
                projection=False, perturbed=False):
    p = S.shape[0]
    e0 = np.zeros((p,))
    e0[0] = 1
    a1 = e0.copy()
    a2 = np.zeros((p-1,))
    mu1 = np.zeros((p-1,))
    if perturbed:
        a1 = a + 0.1*np.random.randn(p)
        a2 = a[1:] + 0.1*np.random.randn(p-1)
    if regularized:
        print('Regularized - ADMM + Newton')
        for i in range(iterations):       
            print('='*50, "iteration ", i, '='*50)
            a1_ = newton_method(f, a1, S, sigma_x, rho, a2, mu1, epsilon=epsilon,
                                alpha=alpha, beta=beta, regularized=regularized,
                                backtracking=backtracking, projection=projection)
            print("Completed Newton Iteration")
            a2_ = soft(a1_[1:]+mu1,lambda_param/rho)
            mu1_ = mu1 + (a1_[1:]-a2_)
            a1, a2, mu1 = a1_, a2_, mu1_
    else:
        print('Unregularized - Newton')
        for i in range(iterations):       
            print('='*50, "iteration ", i, '='*50)
            a1_ = newton_method(f, a1, S, sigma_x, rho, a2, mu1, epsilon=epsilon,
                                alpha=alpha, beta=beta, regularized=regularized,
                                backtracking=backtracking, projection=projection)
            a1 = a1_
    return a1, a2, mu1

### Simulations - Symmetric Case

#### Initialization for multiple simulations

In [62]:
p = 50  # Dimensions of the square toeplitz matrix
d = 5   # Sparsity of the (px1) vextor => no of non zero entries in (2p x 1) = 2d-1
const = 15  # multiplicative factor for number of samples
n_samples = int(const*(d*np.log(p))) # number of samples for the problem setup
num_rep = 10  # number of repetitions of the algorithm

rho = 1
lambda_param = 0.05
iterations = 50

regularize = True
backtrack = True
project = True
perturb = False
n_samples

293

In [63]:
data = {}

print('Dimensions of Matrix = ', p, 'Sparsity = ', d)
for rep in range(num_rep):
    a = create_sparse_vec_pos_def_2(p, d, diag= d*10)
    B0 = B_mat_symmetric(a, p)
    B0_inv = np.linalg.inv(B0)
    sigma_x = cov_x(p)
    sigma_y = cov_y(sigma_x,B0)
    y_samp = generate_y(np.zeros((p,)), sigma_y, n_samples)
    S = samp_cov(y_samp)
    
    total_t1 = time()
    print('_'*50, 'Run = ', rep, '_'*50)
    a1, a2, mu1 = ADMM_newton(f1, iterations, rho, lambda_param, S, sigma_x, a, epsilon=1e-4,
                              alpha=0.1, beta=0.7, regularized=regularize, backtracking=backtrack,
                              projection=project, perturbed=perturb)
    total_t2 = time()
    print('Total time taken for a run = ', total_t2 - total_t1)
    
    run_data = {'p': p, 'd':d, 'const':const, 'num_samples':n_samples, 'lambda':lambda_param, 'rho': rho,
                'iterations': iterations, 'a': a, 'samp_y':y_samp, 'f_reg':regularize, 'f_back':backtrack,
                'f_proj':project, 'f_perturb':perturb, 'a1':a1, 'a2':a2, 'mu1':mu1}
    data[rep] = run_data

Dimensions of Matrix =  50 Sparsity =  5
__________________________________________________ Run =  0 __________________________________________________
Regularized - ADMM + Newton
================================================== iteration  0 ==================================================
line search
999544.186882299 [ 9.99089362e-01  2.06696319e-06  2.66257788e-06  2.08914489e-04
 -1.92248467e-06 -1.18403120e-05  9.23677989e-05 -5.19287680e-06
 -9.70256903e-06  2.06783461e-04 -8.70021063e-06 -9.32834606e-06
  6.89530542e-05  3.06150564e-06  1.18660976e-08  1.79324747e-05
  7.57325484e-06 -1.20531078e-05  6.36342456e-05  8.50243691e-06
 -4.11091399e-06  4.26946225e-05 -6.16915321e-06 -3.29407522e-06
  5.84578524e-05  2.32437077e-06 -1.18585577e-05  1.24022834e-04
  6.64560585e-06  2.22781969e-06  8.44325966e-05 -9.54980521e-06
 -1.30180841e-06  5.25672845e-05 -1.02190064e-06 -1.71442172e-05
  3.61475396e-05 -6.06102664e-06 -1.99970641e-05  6.95687122e-06
  1.15461139e-05  9.349911

line search
line search
not much change in x observed
2
Completed Newton Iteration
================================================== iteration  8 ==================================================
line search
0.13772158471387627 [ 1.26900511e-01 -1.44826074e-04 -1.59457845e-04  1.79866090e-01
 -1.03270892e-03 -4.00425032e-02 -2.29453892e-02 -1.60217060e-02
 -2.46194132e-02  1.88547841e-01 -3.03910920e-02 -2.60485660e-02
 -1.87243046e-02 -6.62673092e-05 -1.65929167e-04 -7.49333071e-02
 -7.86976039e-05 -2.70572880e-02 -8.58545553e-03 -5.57126374e-05
 -3.91707060e-03 -1.96100659e-02 -1.48769843e-02 -3.30615654e-03
  4.44406371e-03 -3.94849573e-05 -2.12304813e-02  8.18236105e-02
 -3.05000942e-05 -7.78946238e-05  2.72620139e-02 -1.75807209e-02
 -5.25657310e-05 -4.36604259e-03 -9.18395255e-04 -1.85248102e-02
 -2.24520280e-02 -9.08913322e-03 -2.12479144e-02 -1.39092589e-02
 -5.69640912e-05 -4.14227533e-05 -4.10946341e-03 -9.04502888e-04
 -1.42694161e-02 -1.80173754e-03 -2.70972561e-04 -1.330

line search
line search
line search
not much change in x observed
3
Completed Newton Iteration
================================================== iteration  16 ==================================================
line search
0.04068561586475622 [ 4.04102190e-02 -2.52930652e-05 -3.06010248e-05  5.74613478e-02
 -1.43109677e-04 -3.99548704e-02 -3.80531761e-02 -1.58531348e-02
 -2.30594995e-02  6.95674427e-02 -3.09592904e-02 -2.62763587e-02
 -2.69980350e-02 -1.42161473e-05 -3.22941598e-05 -8.02896250e-02
 -2.22974707e-04 -2.61562132e-02 -2.07812478e-02 -1.37636018e-05
 -3.17704638e-03 -2.46179450e-02 -1.46873934e-02 -3.03033042e-03
 -5.32147793e-03 -1.18002949e-04 -2.07895111e-02  4.09755611e-02
 -6.37486154e-06 -1.36865151e-05  8.60088957e-03 -1.77963425e-02
 -7.06435739e-06 -7.56977425e-03 -5.83782088e-04 -1.74428314e-02
 -3.01099472e-02 -9.34377860e-03 -2.14554215e-02 -1.75112873e-02
 -2.29518780e-06 -8.94443960e-06 -4.48196239e-03 -1.30930733e-03
 -1.44398090e-02 -2.00654722e-03 -3.156568

line search
line search
not much change in x observed
2
Completed Newton Iteration
================================================== iteration  24 ==================================================
line search
0.02694807052271772 [ 1.63639392e-02 -3.22947917e-06 -5.92414392e-06  2.34177502e-02
 -4.43551935e-06 -4.00318407e-02 -2.27003831e-02 -1.58714381e-02
 -2.25573422e-02  3.04891199e-02 -3.11691374e-02 -2.65938756e-02
 -1.65649046e-02 -4.00125190e-06 -6.72889126e-06 -7.41877706e-02
 -3.29874592e-04 -2.59035174e-02 -1.52561995e-02 -1.63396954e-06
 -3.07764480e-03 -2.28867984e-02 -1.45388399e-02 -3.09309333e-03
 -4.99710767e-03 -1.81462253e-04 -2.06943680e-02  2.26595540e-02
 -3.62703650e-06 -1.86738890e-06  2.22638080e-03 -1.78230852e-02
 -6.20219328e-05 -8.06719283e-03 -3.89410569e-04 -1.70152486e-02
 -3.30512149e-02 -9.37131797e-03 -2.15786720e-02 -1.84757833e-02
 -1.55546179e-06 -2.47760856e-06 -4.27984748e-03 -1.47902000e-03
 -1.45291673e-02 -1.91055608e-03 -3.23976869e-04 -7.80

line search
line search
not much change in x observed
2
Completed Newton Iteration
================================================== iteration  32 ==================================================
line search
0.025168157446763238 [ 7.45423504e-03 -3.27398541e-06 -3.76203437e-06  9.78681074e-03
 -3.48318807e-06 -3.99994335e-02 -1.22323718e-02 -1.58429708e-02
 -2.23015035e-02  1.22032832e-02 -3.12139598e-02 -2.67887515e-02
 -2.09867551e-02 -2.52888147e-06 -3.56429277e-06 -7.31015878e-02
 -3.99955482e-04 -2.57447757e-02 -1.65392523e-02 -3.20965599e-06
 -3.01981419e-03 -2.34019737e-02 -1.43994049e-02 -3.07464860e-03
 -4.05585144e-03 -2.09271446e-04 -2.06081452e-02  1.24548036e-02
 -3.36173534e-06 -1.84350271e-06 -3.61641951e-04 -1.78779565e-02
 -7.41483195e-05 -8.32996797e-03 -2.90787152e-04 -1.68195294e-02
 -3.46884058e-02 -9.38584288e-03 -2.16121160e-02 -1.91473442e-02
 -5.42198422e-07 -1.08358164e-06 -4.32483038e-03 -1.53011409e-03
 -1.45730606e-02 -1.97321115e-03 -3.30018480e-04 -5.9

...

line search
line search
not much change in x observed
2
Completed Newton Iteration
================================================== iteration  16 ==================================================
line search
0.038276072506890045 [ 4.21670773e-02 -8.76415693e-03 -8.33679145e-03  5.52256577e-02
 -9.52185132e-06 -1.76992812e-02 -1.25596045e-02 -1.33031628e-02
 -2.83024008e-03 -1.58426737e-02 -1.33489758e-05  3.26548836e-02
 -1.19655427e-03 -4.34119918e-02 -2.33177655e-02 -5.74540790e-03
 -6.05628860e-03 -5.43528562e-02 -4.52009832e-03 -3.95711794e-02
 -2.30867361e-02 -7.84199270e-03 -2.64003070e-02 -4.41107669e-03
 -1.12806693e-05  1.04080857e-01  1.85430284e-03 -9.72936340e-03
 -2.04187736e-02 -1.13641474e-02 -5.30650699e-03 -2.30396419e-02
 -5.22296690e-03 -2.43233210e-02 -1.76229137e-03 -5.07750014e-06
 -3.04675866e-02 -4.64731223e-03 -1.22304931e-02 -1.07839419e-02
 -1.58491472e-03 -1.37541024e-02 -1.99241872e-03 -5.63959656e-04
 -7.81754768e-03 -8.19138842e-03 -2.71909083e-04 -1.9

line search
line search
not much change in x observed
2
Completed Newton Iteration
================================================== iteration  24 ==================================================
line search
0.024705373861434395 [ 1.96419783e-02 -1.00620779e-02 -8.07062124e-03  1.92984367e-02
 -8.84604767e-06 -1.81668719e-02 -6.78386773e-03 -1.35011807e-02
 -4.19554983e-03 -1.50487797e-02 -1.09930153e-05  1.36618712e-02
 -4.19229833e-04 -4.38195822e-02 -1.75182865e-02 -6.46984686e-03
 -5.25553556e-03 -5.41717997e-02 -4.50519830e-03 -3.93831754e-02
 -2.27140394e-02 -7.99735506e-03 -2.70202698e-02 -4.74399215e-03
 -5.28654830e-06  5.68452849e-02  1.11377511e-03 -1.00859471e-02
 -2.31134647e-02 -1.18308355e-02 -5.28279253e-03 -2.28986132e-02
 -5.38007194e-03 -2.59566457e-02 -1.59968027e-03 -2.50556462e-06
 -3.47046965e-02 -4.98459529e-03 -1.20974871e-02 -1.07182151e-02
 -1.60550670e-03 -1.38999585e-02 -1.74581584e-03 -6.02799737e-04
 -7.85524430e-03 -8.20352943e-03 -2.82984871e-04 -1.9

line search
line search
not much change in x observed
2
Completed Newton Iteration
================================================== iteration  32 ==================================================
line search
0.022561829652464088 [ 8.97309842e-03 -1.07681423e-02 -7.89371636e-03  6.67981835e-03
 -2.84851463e-06 -1.78403859e-02 -3.02014359e-03 -1.36193921e-02
 -2.65252261e-03 -1.43328036e-02 -5.05127260e-05  6.51716469e-03
  3.81225336e-05 -4.40633315e-02 -1.09211979e-02 -6.79184944e-03
 -4.93323839e-03 -5.36563281e-02 -4.52267636e-03 -3.99881450e-02
 -2.25786022e-02 -8.08344024e-03 -3.31984246e-02 -4.94206333e-03
 -1.72025286e-06  3.02050445e-02  6.68360370e-04 -1.03190034e-02
 -2.82158697e-02 -1.20267574e-02 -5.53004542e-03 -2.33261867e-02
 -5.43247618e-03 -2.71399665e-02 -1.62067157e-03 -1.00465751e-06
 -3.70887715e-02 -5.13763693e-03 -1.20244809e-02 -1.06810379e-02
 -1.58421854e-03 -1.40036587e-02 -1.61673773e-03 -5.90516981e-04
 -7.92352792e-03 -8.21633553e-03 -2.71467564e-04 -2.0

line search
line search
not much change in x observed
2
Completed Newton Iteration
================================================== iteration  40 ==================================================
line search
0.022376536825835857 [ 4.43190505e-03 -1.11446003e-02 -7.79598662e-03  2.46450871e-03
 -1.40257475e-06 -1.75572829e-02 -1.16089644e-03 -1.36708434e-02
 -1.37410818e-03 -1.39516351e-02 -8.90043117e-05  3.47855067e-03
 -3.55716844e-06 -4.41781541e-02 -6.25936310e-03 -6.95292953e-03
 -4.75927814e-03 -5.31560647e-02 -4.53286921e-03 -4.02462555e-02
 -2.24693920e-02 -8.12157161e-03 -3.61891887e-02 -5.04123217e-03
 -7.36112451e-07  1.61563761e-02  4.06150783e-04 -1.04368627e-02
 -3.07168731e-02 -1.21221106e-02 -5.65829859e-03 -2.35409110e-02
 -5.45076652e-03 -2.77218320e-02 -1.64287826e-03 -4.43059386e-07
 -3.83073385e-02 -5.20540438e-03 -1.19767961e-02 -1.05742834e-02
 -1.56195379e-03 -1.40640805e-02 -1.52766547e-03 -5.79102660e-04
 -7.95763307e-03 -8.22144681e-03 -2.63691781e-04 -2.1

line search
line search
not much change in x observed
2
Completed Newton Iteration
================================================== iteration  48 ==================================================
line search
0.022540598082379123 [ 2.27710049e-03 -1.13522206e-02 -7.75663914e-03  9.50960296e-04
 -7.05485630e-07 -1.74124631e-02 -3.98305375e-04 -1.36952201e-02
 -6.81596068e-04 -1.37824317e-02 -1.17380219e-04  1.92750353e-03
 -1.86155163e-06 -4.42392957e-02 -3.48655054e-03 -7.04101854e-03
 -4.67881103e-03 -5.28609968e-02 -4.54167126e-03 -4.04007515e-02
 -2.24129666e-02 -8.13960142e-03 -3.77133547e-02 -5.09744868e-03
 -3.51474280e-07  8.67063490e-03  2.45856764e-04 -1.04996229e-02
 -3.19918517e-02 -1.21742644e-02 -5.73106112e-03 -2.36694528e-02
 -5.45926832e-03 -2.80300066e-02 -1.66357641e-03 -2.15382844e-07
 -3.89552117e-02 -5.23988311e-03 -1.19488678e-02 -1.04935426e-02
 -1.54712285e-03 -1.40994566e-02 -1.47533746e-03 -5.71252744e-04
 -7.97606563e-03 -8.22423430e-03 -2.59156566e-04 -2.1

In [64]:
keys = list(data[0].keys())
best_thr = []
best_rate = []
best_acc = []
best_f1 = []
best_cm = []

kll_const = []
rkll_const = []
rte_const = []
norm_const = []
scores_const = []

threshold_range = 0.01*np.arange(101)
for rep in range(num_rep):
    a = data[rep][keys[7]]
    a1 = data[rep][keys[-3]]
    true_a_binary = binarize_matrix(a)
    thr_rate = []
    thr_cm = []
    thr_acc = []
    thr_f1 = []
    for thr in threshold_range:  
        a_thr = threshold(a1, threshold_value=thr) # Apply the threshold
        a_binary = binarize_matrix(a_thr)  # Change the output to a binary matrix to check for support recovery
        rate = compute_recovery_rate_numpy(a_binary, true_a_binary)  # Computing recovery rate
        cm = confusion_matrix(true_a_binary, a_binary)
        accuracy = (cm[0,0] + cm[1,1])/np.sum(cm)

        thr_rate += [rate]
        thr_cm += [cm]
        thr_acc += [accuracy]
        thr_f1 += [f1_score(true_a_binary, a_binary)]
    idx = np.argmax(np.array(thr_acc))
    best_thr += [threshold_range[idx]]
    best_rate += [thr_rate[idx]]
    best_acc += [thr_acc[idx]]
    best_f1 += [thr_f1[idx]]
    best_cm += [thr_cm[idx]]
    a_thr = threshold(a1, threshold_value=threshold_range[idx]) # Apply the threshold
    a_binary = binarize_matrix(a_thr)  # Change the output to a binary matrix to check for support recovery
    
    kll_const += [kll(a, a_thr, sigma_x)]
    rkll_const += [rkll(a, a_thr, sigma_x)]
    rte_const += [rte(a, a_thr, sigma_x)]
    norm_const += [norm_loss(a, a_thr, sigma_x)]
    scores_const += [scores(a, a1, threshold_value=threshold_range[idx])]

In [65]:
fro_norm_const = [norm_const[i][0] for i in range(num_rep)]
spec_norm_const = [norm_const[i][1] for i in range(num_rep)]
l1_norm_const = [norm_const[i][2] for i in range(num_rep)]

In [67]:
[average(best_acc), average(best_f1), average(best_rate), average(kll_const), average(rkll_const), average(rte_const)]

[0.998,
 0.9888888888888889,
 0.9800000000000001,
 0.11453063443939086,
 0.11240762036352692,
 0.013115767773690955]